<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Sequence_to_Sequence_Models_(Seq2Seq).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        outputs, (hidden, cell) = self.rnn(embedded)
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, n_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hidden_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, hidden, cell):
        trg = trg.unsqueeze(0)
        embedded = self.dropout(self.embedding(trg))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg):
        hidden, cell = self.encoder(src)
        output, hidden, cell = self.decoder(trg[0], hidden, cell)
        return output

# Hyperparameters
INPUT_DIM = 100
OUTPUT_DIM = 100
ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
HIDDEN_DIM = 64
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

# Instantiate the encoder and decoder
encoder = Encoder(INPUT_DIM, ENC_EMB_DIM, HIDDEN_DIM, N_LAYERS, ENC_DROPOUT)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HIDDEN_DIM, N_LAYERS, DEC_DROPOUT)

# Create Seq2Seq model
model = Seq2Seq(encoder, decoder)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
encoder_optimizer = optim.Adam(encoder.parameters(), lr=0.01)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=0.01)

# Training loop for Seq2Seq model
def train(seq2seq_model, iterator, criterion, enc_optimizer, dec_optimizer, clip):
    seq2seq_model.train()
    epoch_loss = 0

    for batch in iterator:
        src, trg = batch
        enc_optimizer.zero_grad()
        dec_optimizer.zero_grad()

        output = seq2seq_model(src, trg)

        loss = criterion(output, trg[1])
        loss.backward()

        torch.nn.utils.clip_grad_norm_(seq2seq_model.parameters(), clip)

        enc_optimizer.step()
        dec_optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

# Example usage with dummy data
dummy_iterator = [(
    torch.randint(0, INPUT_DIM, (10,)),
    torch.randint(0, OUTPUT_DIM, (10,))
) for _ in range(100)]

# Train for one epoch
train_loss = train(model, dummy_iterator, criterion, encoder_optimizer, decoder_optimizer, clip=1)
print(f"Training Loss: {train_loss:.4f}")